# How to get MOC data from VESPA in Python

Illustrate search for TAP MOC for vespa using 'pyvo' and 'astropy'

## Import required modules

In [14]:
import pyvo as vo
from mpl_toolkits.basemap import Basemap
import re

import astropy.units as u
from astropy.coordinates import SkyCoord

from astroquery.cds import cds
from astroquery.vizier import Vizier

from ipyaladin import Aladin

## Search a specific TAP service

We are looking for VVEx - VIRTIS/Venus Express database since it has a MOC

### Search the database containing this service

In [232]:
keywords = ["Venus"]

In [221]:
databasesList = vo.registry.search(servicetype='tap',keywords=keywords, includeaux=True)
print(databasesList.to_table())

                        ivoid                          ... intf_roles
                                                       ...           
------------------------------------------------------ ... ----------
                         ivo://cds.vizier/j/a+a/537/l2 ...        std
                        ivo://cds.vizier/j/a+a/585/a53 ...        std
                        ivo://cds.vizier/j/a+a/606/a53 ...        std
                        ivo://cds.vizier/j/a+a/613/a25 ...        std
                        ivo://cds.vizier/j/a+a/619/a99 ...        std
                        ivo://cds.vizier/j/a+a/627/a82 ...        std
                                                   ... ...        ...
ivo://pds-ppi/mess_mag_calibrated_vso_avg/ppi/epn_core ...        std
          ivo://pds-ppi/pvo_ephem_vso_asc/ppi/epn_core ...        std
          ivo://pds-ppi/pvo_ephem_vso_bin/ppi/epn_core ...        std
    ivo://pds-ppi/vex_aspera4_els_derived/ppi/epn_core ...        std
        ivo://pds-pp

#### Print the full list of databases found

In [246]:
for d in databasesList :
    print(d.short_name)

J/A+A/537/L2
J/A+A/585/A53
J/A+A/606/A53
J/A+A/613/A25
J/A+A/619/A99
J/A+A/627/A82
J/A+A/653/A41
J/AJ/154/207
J/AJ/158/196
J/AJ/160/131
J/AJ/160/259
J/AJ/162/216
J/ApJ/800/135
J/ApJ/807/45
J/ApJ/893/L27
J/MNRAS/462/4300
J/MNRAS/485/3999
J/MNRAS/519/6210
VI/127

GRB/IPN
KONUS
Cassini RPWS
BDIP
bdip.observatori
HST Planeto
VVEx
MESS-MAG-VSO
MESS-MAG-VSO-AVG
PVO-PHEM-ASC
PVO-PHEM-BIN
VEX-ASPERA4-DER
VEX-ASPERA4-ENG
VEX-ASPERA4-RAW
VEX-HIGH-MAG


### Search the service in the databases that were found

table_limit is too low by default so it needs to be specified here

In [298]:
services = []
progress = 0
for d in databasesList :
    print("Searching ("+str(progress)+"/"+str(len(databasesList))+")")
    for key in list(d.get_tables(table_limit=50).keys()) :
        s = d.get_tables(table_limit=50)[key]
        if re.search("^.*"+nameServiceSearched.upper()+".*$", s.description.upper() + " " + s.title.upper() + " " + s.name.upper()) :
            services.append({"service":s, "database":d})  
    progress = progress + 1

Searching (0/35)
Searching (1/35)
Searching (2/35)
Searching (3/35)
Searching (4/35)
Searching (5/35)
Searching (6/35)
Searching (7/35)
Searching (8/35)
Searching (9/35)
Searching (10/35)
Searching (11/35)
Searching (12/35)
Searching (13/35)
Searching (14/35)
Searching (15/35)
Searching (16/35)
Searching (17/35)
Searching (18/35)
Searching (19/35)
Searching (20/35)
Searching (21/35)
Searching (22/35)
Searching (23/35)
Searching (24/35)
Searching (25/35)
Searching (26/35)
Searching (27/35)
Searching (28/35)
Searching (29/35)
Searching (30/35)
Searching (31/35)
Searching (32/35)
Searching (33/35)
Searching (34/35)


In [300]:
for s in services :
    print(s["service"].name," in ", s["database"]["short_name"])

J/A+A/537/L2/tablea1  in  J/A+A/537/L2
J/A+A/627/A82/tablea3  in  J/A+A/627/A82
J/A+A/627/A82/tablea1  in  J/A+A/627/A82
J/A+A/627/A82/tablea2  in  J/A+A/627/A82
J/AJ/162/216/fig11  in  J/AJ/162/216
vvex.epn_core  in  VVEx


## Only keep tables with a 'coverage' key (and so maybe a MOC)

In [301]:
def doContainMOC(table) :
    if table == None :
        return False
    for field in table.columns :
        if re.search("^.*<BaseParam name=\"coverage\"/>.*$", str(field)) :
            return True
    return False

In [302]:
servicesWithMOC = []
for s in services :
    if doContainMOC(s["service"]) :
        servicesWithMOC.append(s)

In [304]:
for s in servicesWithMOC :
    print(s["service"].name)

vvex.epn_core


## Display the first 3 elements in the table
Here we only consider the first service found in the previous section

In [284]:
def getData(tapService, serviceStructure, query) :
    if doContainMOC(serviceStructure) :
        return tapService.search(query).to_table()
    else :
        return "NO MOC DATA"

In [307]:
service = servicesWithMOC[0]["service"]
database = servicesWithMOC[0]["database"]
print(service.name, " in ", database["short_name"])

vvex.epn_core  in  VVEx
<class 'pyvo.io.vosi.vodataservice.Table'>


In [309]:
tapService = vo.dal.TAPService(database.access_url)

data = tapService.search("""SELECT TOP 10 * FROM """ + service.name).to_table()
print("\nAll data: ",data)

data = tapService.search("""SELECT TOP 10 coverage FROM """ + service.name).to_table()
print("\nMOC: ",data)


All data:  granule_uid granule_gid   obs_id  ... science_case_id sc_pointing_mode
                                  ...                                 
----------- ----------- --------- ... --------------- ----------------
 VV1292_05C  calibrated VV1292_05 ...               7             NULL
 VV1292_05G    geometry VV1292_05 ...               7             NULL
 VV1821_05C  calibrated VV1821_05 ...               0            INERT
 VV1821_05G    geometry VV1821_05 ...               0            INERT
 VV1290_07C  calibrated VV1290_07 ...               7            INERT
 VV1290_07G    geometry VV1290_07 ...               7            INERT
 VV1291_01C  calibrated VV1291_01 ...               0             NULL
 VV1291_01G    geometry VV1291_01 ...               0             NULL
 VV1291_02C  calibrated VV1291_02 ...               0             NULL
 VV1291_02G    geometry VV1291_02 ...               0             NULL

MOC:                                                            

## ...


In [310]:
data = getData(vo.dal.TAPService(database.access_url), service, """SELECT TOP 100 * FROM """ + service.name)

In [311]:
aladin = Aladin(target=data[0]["granule_uid"])
aladin

Aladin(options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'overlay_sur…

In [312]:
aladin.add_table(data)
